In [6]:
import pandas as pd

In [15]:
df = pd.read_csv('data/k_d_01_2020.csv',encoding = 'unicode_escape')
df.iloc[:10]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14
0,249180010,PSZCZYNA,2020,1,1,2.4,-0.2,1.0,-4.4,0.0,9.0,NaN,0,9.0
1,249180010,PSZCZYNA,2020,1,2,4.4,-3.6,-0.5,-6.9,0.0,9.0,NaN,0,9.0
2,249180010,PSZCZYNA,2020,1,3,5.3,-5.2,0.4,-7.9,0.0,9.0,NaN,0,9.0
3,249180010,PSZCZYNA,2020,1,4,3.6,0.8,1.6,-3.5,1.4,NaN,S,0,9.0
4,249180010,PSZCZYNA,2020,1,5,2.3,-2.2,-0.6,-5.9,0.0,9.0,NaN,2,NaN
5,249180010,PSZCZYNA,2020,1,6,1.0,-4.7,-1.9,-8.9,0.0,9.0,NaN,1,NaN
6,249180010,PSZCZYNA,2020,1,7,3.4,-7.2,-2.8,-10.9,0.9,NaN,W,1,NaN
7,249180010,PSZCZYNA,2020,1,8,5.3,-1.4,1.8,-2.5,1.8,NaN,W,1,NaN
8,249180010,PSZCZYNA,2020,1,9,7.3,2.0,4.3,-2.5,1.8,NaN,W,0,9.0
9,249180010,PSZCZYNA,2020,1,10,5.7,1.3,3.3,-1.0,0.8,NaN,W,0,9.0


In [14]:
df.loc[df["t2"]=="OLECKO"].iloc[:10]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14
1953,254220090,OLECKO,2020,1,1,4.1,-1.3,1.2,-1.6,0.3,NaN,W,0,9.0
1954,254220090,OLECKO,2020,1,2,2.7,-1.5,0.7,-1.4,0.0,9.0,NaN,0,9.0
1955,254220090,OLECKO,2020,1,3,2.1,-2.8,-0.3,-3.4,1.8,NaN,W,0,9.0
1956,254220090,OLECKO,2020,1,4,3.9,0.1,1.7,-0.1,1.9,NaN,S,0,9.0
1957,254220090,OLECKO,2020,1,5,1.3,-2.8,-1.2,-4.4,0.6,NaN,S,0,NaN
1958,254220090,OLECKO,2020,1,6,0.4,-4.4,-1.2,-5.6,0.6,NaN,S,0,NaN
1959,254220090,OLECKO,2020,1,7,0.6,0.0,0.2,-0.2,0.0,9.0,NaN,0,NaN
1960,254220090,OLECKO,2020,1,8,2.5,-2.3,0.7,-2.9,2.8,NaN,W,0,9.0
1961,254220090,OLECKO,2020,1,9,6.2,2.2,3.9,1.2,4.1,NaN,W,0,9.0
1962,254220090,OLECKO,2020,1,10,7.3,3.2,5.7,2.9,9.1,NaN,W,0,9.0
